In [ ]:
# This script tries to model the cost of siphash

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import math
from scipy import optimize
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

figsize=(4.5, 3)

AnonBlue='#3070b3'
AnonGreen='#a2ad00'
AnonOrange='#e37222'

PATH = "../../paper/graphs"

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def sym_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / ((y_true + y_pred) / 2))) * 100

In [ ]:
data = pd.read_csv("../data-files/siphash.csv")

In [ ]:
# The siphash model
def fitfunc(x,t,o):
    # Actual mathematical model
    def runner(a):
        return (t/a) + o
    # Magic for scipy
    if isinstance(x, np.ndarray):
        ret = []
        for s in x:
            ret += [runner(s)]
        return ret
    else:
        return runner(x)

#params = [1.02,2**11,600]
# Fit the data
params, _ = optimize.curve_fit(fitfunc, data['size'], data['cycles'])

print(params)

# Plot the data
fit = []
for s in data['size']:
    fit += [fitfunc(s,params[0],params[1])]

r2 = r2_score(data['cycles'], fit) 
print("R2 score: " + str(r2))

mae = mean_absolute_error(data['cycles'], fit)
print("MAE score: " + str(mae))

mape = mean_absolute_percentage_error(data['cycles'], fit)
print("MAPE score: " + str(mape))

smape = sym_mean_absolute_percentage_error(data['cycles'], fit)
print("sMAPE score: " + str(smape))

fig = plt.figure(figsize=figsize, dpi=300)

plt.plot(data['size'], fit, color=AnonOrange, label="fit", zorder=2, linewidth=2, linestyle='--', dashes=(5, 1))
plt.scatter(data['size'], data['cycles'], 5, label="data", color=AnonBlue, zorder=1)

plt.ylim([1,2**10])
#plt.semilogx(basex=2)
plt.loglog(basex=2, basey=2)
plt.grid(b=True)
plt.xlabel('\#Operations')
plt.ylabel('Cycles per Op')
plt.legend()
#fig.savefig('output/synth-siphash.pdf',bbox_inches='tight')
#plt.show()

from matplotlib2tikz import save as tikz_save
tikz_save(PATH + '/siphash.tex',
          figureheight='3.5cm',
          figurewidth='.9\\columnwidth',
          extra_axis_parameters = {"log basis x=2",
                                   "mark options={scale=.15}",
                                   "legend pos = south east"}
         )